In [5]:
import os
import csv

parent_folder: str = "Datasets/train/"
target_folder: str = "Datasets/train/"

classes = ['angry', 'disgust', 'happy', 'surprise']
emotions: list[str] = [folder_name for folder_name in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, folder_name)) and folder_name in classes]

data: list[str] = []

for emotion in emotions :
    folder_path = os.path.join(parent_folder, emotion)
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            data.append([os.path.join(target_folder, emotion, filename), emotion])

if not os.path.exists('./CSV/') :
    os.makedirs('./CSV/')

with open("./CSV/emotions_train.csv", "w", newline="") as csv_file:
    writer: csv = csv.writer(csv_file)
    writer.writerow(["filepath", "emotion"])
    writer.writerows(data)

In [6]:
import os
import csv

parent_folder: str = "Datasets/test/"
target_folder: str = "Datasets/test/"

classes = ['angry', 'disgust', 'happy', 'surprise']
emotions: list[str] = [folder_name for folder_name in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, folder_name)) and folder_name in classes]

data: list = []

for emotion in emotions:
    folder_path = os.path.join(parent_folder, emotion)
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            data.append([os.path.join(target_folder, emotion, filename), emotion])

if not os.path.exists('./CSV/') :
    os.makedirs('./CSV/')

with open("./CSV/emotions_test.csv", "w", newline="") as csv_file:
    writer: csv = csv.writer(csv_file)
    writer.writerow(["filepath", "emotion"])
    writer.writerows(data)

In [7]:
import tensorflow as tf
import pandas as pd
import csv
import os
from PIL import Image

img_size = 48
batch_size = 64

classes = ['angry', 'disgust', 'happy', 'surprise']

df = pd.read_csv('CSV/emotions_train.csv')

with open('CSV/train/processed_train.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    header = ['emotion'] + [f'pixel{i}' for i in range(img_size * img_size)]
    writer.writerow(header)

    for index, row in df.iterrows():
        file_path = row['filepath']
        label = row['emotion']

        img = Image.open(file_path)
        img = img.resize((img_size, img_size))

        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)

        num_images_to_generate = 1
        datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
        if label == 'disgust':
            datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255,
                rotation_range=20,
                zoom_range=0.15,
                width_shift_range=0.2,
                height_shift_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest'
            )
            num_images_to_generate = 7

        data_generator = datagen.flow(img_array, batch_size=1)

        for _ in range(num_images_to_generate):
            x_batch = next(data_generator)
            x_flat = x_batch.reshape(x_batch.shape[0], -1)
            row = [label] + list(x_flat[0])
            writer.writerow(row)


In [8]:
import tensorflow as tf
import pandas as pd
import csv
import os
from PIL import Image

img_size = 48
batch_size = 64

classes = ['angry', 'disgust', 'happy', 'surprise']

df = pd.read_csv('CSV/emotions_test.csv')

with open('CSV/test/processed_test.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    header = ['emotion'] + [f'pixel{i}' for i in range(img_size * img_size)]
    writer.writerow(header)

    for index, row in df.iterrows():
        file_path = row['filepath']
        label = row['emotion']

        img = Image.open(file_path)
        img = img.resize((img_size, img_size))

        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)

        num_images_to_generate = 1
        datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
        if label == 'disgust':
            datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255,
                rotation_range=20,
                zoom_range=0.15,
                width_shift_range=0.2,
                height_shift_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest'
            )
            num_images_to_generate = 12

        data_generator = datagen.flow(img_array, batch_size=1)

        for _ in range(num_images_to_generate):
            x_batch = next(data_generator)
            x_flat = x_batch.reshape(x_batch.shape[0], -1)
            row = [label] + list(x_flat[0])
            writer.writerow(row)
